# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [51]:
# import libraries
import pandas as pd 
import numpy as np
from sqlalchemy import create_engine 

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
import re

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

nltk.download(['punkt', 'punkt_tab', 'wordnet', 'averaged_perceptron_tagger'])

import warnings
import pickle

warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\z635177\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\z635177\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\z635177\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\z635177\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [20]:
# load data from database
engine = create_engine('sqlite:///../data/DisasterResponse.db')
df = pd.read_sql("SELECT * FROM disaster_reponse_tbl", con = engine)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26216 entries, 0 to 26215
Data columns (total 40 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id                      26216 non-null  int64 
 1   message                 26216 non-null  object
 2   original                10170 non-null  object
 3   genre                   26216 non-null  object
 4   related                 26216 non-null  int64 
 5   request                 26216 non-null  int64 
 6   offer                   26216 non-null  int64 
 7   aid_related             26216 non-null  int64 
 8   medical_help            26216 non-null  int64 
 9   medical_products        26216 non-null  int64 
 10  search_and_rescue       26216 non-null  int64 
 11  security                26216 non-null  int64 
 12  military                26216 non-null  int64 
 13  child_alone             26216 non-null  int64 
 14  water                   26216 non-null  int64 
 15  fo

In [22]:
df.describe()

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
count,26216.00000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,...,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000,26216.000000
mean,15224.82133,0.773650,0.170659,0.004501,0.414251,0.079493,0.050084,0.027617,0.017966,0.032804,...,0.011787,0.043904,0.278341,0.082202,0.093187,0.010757,0.093645,0.020217,0.052487,0.193584
std,8826.88914,0.435276,0.376218,0.066940,0.492602,0.270513,0.218122,0.163875,0.132831,0.178128,...,0.107927,0.204887,0.448191,0.274677,0.290700,0.103158,0.291340,0.140743,0.223011,0.395114
min,2.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7446.75000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15662.50000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,22924.25000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,30265.00000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [23]:
#attribute child_alone has no variation
df.drop("child_alone", axis = 1, inplace = True)

In [24]:
#adapt error for feature "related" & "direct_report"
df["related"] = df["related"].map(lambda x: 1 if x==2 else x)
df["direct_report"] = df["direct_report"].map(lambda x: 1 if x==2 else x)

In [25]:
X = df["message"]
y = df[df.columns[4:]].astype("int")

### 2. Write a tokenization function to process your text data

In [26]:
def tokenize(text):
    
    #replace all urls
    url_regex = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")

    #create tokens & lemmatize
    tokens = word_tokenize(text)
    
    #lemmatize, Normalize
    lemmatizer = WordNetLemmatizer()
        
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    
    
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [33]:
def create_pipeline(clf):

    pipeline = Pipeline([
        ("features", FeatureUnion([
            ("text_pipeline", Pipeline([
                ("vect", CountVectorizer(tokenizer=tokenize)),
                ("tfidf", TfidfTransformer())
            ]))
        ],n_jobs=-1)),
        ("clf", MultiOutputClassifier(clf,n_jobs=-1))
    ])

    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [35]:
pipeline_model = create_pipeline(clf = RandomForestClassifier())
pipeline_model.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(n_jobs=-1,
                              transformer_list=[('text_pipeline',
                                                 Pipeline(steps=[('vect',
                                                                  CountVectorizer(tokenizer=<function tokenize at 0x000001170582BEC0>)),
                                                                 ('tfidf',
                                                                  TfidfTransformer())]))])),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(),
                                       n_jobs=-1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [36]:
y_pred_test = pipeline_model.predict(X_test)

In [37]:
print(classification_report(y_test.values, y_pred_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.81      0.98      0.89      5022
               request       0.90      0.43      0.59      1129
                 offer       0.00      0.00      0.00        23
           aid_related       0.77      0.61      0.68      2722
          medical_help       0.69      0.04      0.07       533
      medical_products       0.72      0.06      0.11       305
     search_and_rescue       0.83      0.03      0.05       180
              security       0.00      0.00      0.00       108
              military       0.60      0.03      0.06       208
                 water       0.87      0.19      0.31       430
                  food       0.84      0.45      0.58       732
               shelter       0.87      0.30      0.45       607
              clothing       0.89      0.08      0.14       102
                 money       1.00      0.03      0.07       173
        missing_people       1.00      

### 6. Improve your model
Use grid search to find better parameters. 

In [46]:
model_pipeline_2 = create_pipeline(clf = RandomForestClassifier())

parameters = {
    "clf__estimator__n_estimators": [100, 500],
    "clf__estimator__max_depth": [3, 10, 15]
}

cv = GridSearchCV(model_pipeline_2, param_grid=parameters, cv = 5, n_jobs=-1, verbose = 3)

cv.fit(X_train, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('features',
                                        FeatureUnion(n_jobs=-1,
                                                     transformer_list=[('text_pipeline',
                                                                        Pipeline(steps=[('vect',
                                                                                         CountVectorizer(tokenizer=<function tokenize at 0x00000117054EF380>)),
                                                                                        ('tfidf',
                                                                                         TfidfTransformer())]))])),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier(),
                                                              n_jobs=-1))]),
             n_jobs=-1,
             param_grid={'clf__estimator__max_depth': [3, 10, 15],
                         'clf__estimator__n_estimators': [100, 500]},
             verbose=3)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [47]:
y_pred = cv.predict(X_test)
print(classification_report(y_test.values, y_pred_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.81      0.98      0.89      5022
               request       0.90      0.43      0.59      1129
                 offer       0.00      0.00      0.00        23
           aid_related       0.77      0.61      0.68      2722
          medical_help       0.69      0.04      0.07       533
      medical_products       0.72      0.06      0.11       305
     search_and_rescue       0.83      0.03      0.05       180
              security       0.00      0.00      0.00       108
              military       0.60      0.03      0.06       208
                 water       0.87      0.19      0.31       430
                  food       0.84      0.45      0.58       732
               shelter       0.87      0.30      0.45       607
              clothing       0.89      0.08      0.14       102
                 money       1.00      0.03      0.07       173
        missing_people       1.00      

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [49]:
model_pipeline_2 = create_pipeline(clf = AdaBoostClassifier())
model_pipeline_2.fit(X_train, y_train)

y_pred = model_pipeline_2.predict(X_test)

print(classification_report(y_test.values, y_pred_test, target_names=y.columns.values))

                        precision    recall  f1-score   support

               related       0.81      0.98      0.89      5022
               request       0.90      0.43      0.59      1129
                 offer       0.00      0.00      0.00        23
           aid_related       0.77      0.61      0.68      2722
          medical_help       0.69      0.04      0.07       533
      medical_products       0.72      0.06      0.11       305
     search_and_rescue       0.83      0.03      0.05       180
              security       0.00      0.00      0.00       108
              military       0.60      0.03      0.06       208
                 water       0.87      0.19      0.31       430
                  food       0.84      0.45      0.58       732
               shelter       0.87      0.30      0.45       607
              clothing       0.89      0.08      0.14       102
                 money       1.00      0.03      0.07       173
        missing_people       1.00      

### 9. Export your model as a pickle file

In [54]:
#with open('classifier.pkl', 'wb') as f:
#    pickle.dump(cv, f)

### 10. Use this notebook to complete `train_classifier.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.